In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('rheeze').getOrCreate()

# Understand the Dataset

In [2]:
spark.read.json('dataset.json') # reading the json

DataFrame[_corrupt_record: string, ageofparticipant: bigint, clinician: struct<branch:string,name:string,role:string>, drug_used: string, experimentenddate: string, experimentstartdate: string, noofhourspassedatfirstreaction: bigint, result: struct<conclusion:string,sideeffectsonparticipant:string>]

In [3]:
trials_df = spark.read.json('dataset.json',multiLine=True)

In [4]:
trials_df.show(5)

+----------------+--------------------+---------+-----------------+-------------------+------------------------------+--------------------+
|ageofparticipant|           clinician|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|              result|
+----------------+--------------------+---------+-----------------+-------------------+------------------------------+--------------------+
|              19|{Ontario, Saul, t...|  Placebo|    1619827200000|      1617235200000|                            52|{BP normalized, r...|
|              14|{Ontario, Saul, n...| Naproxen|    1619827200000|      1617235200000|                            78|    {Follow-up, N/A}|
|              17|{Ontario, Saul, n...|  Placebo|    1619827200000|      1617235200000|                            14|    {Follow-up, N/A}|
|              18|{Ontario, Will, n...| Naproxen|    1619827200000|      1617235200000|                            14|{BP normalized, N/A}|
|              17|{O

In [5]:
trials_df.printSchema()

root
 |-- ageofparticipant: long (nullable = true)
 |-- clinician: struct (nullable = true)
 |    |-- branch: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- role: string (nullable = true)
 |-- drug_used: string (nullable = true)
 |-- experimentenddate: string (nullable = true)
 |-- experimentstartdate: string (nullable = true)
 |-- noofhourspassedatfirstreaction: long (nullable = true)
 |-- result: struct (nullable = true)
 |    |-- conclusion: string (nullable = true)
 |    |-- sideeffectsonparticipant: string (nullable = true)



In [6]:
trials_df.columns

['ageofparticipant',
 'clinician',
 'drug_used',
 'experimentenddate',
 'experimentstartdate',
 'noofhourspassedatfirstreaction',
 'result']

In [7]:
trials_df.dtypes

[('ageofparticipant', 'bigint'),
 ('clinician', 'struct<branch:string,name:string,role:string>'),
 ('drug_used', 'string'),
 ('experimentenddate', 'string'),
 ('experimentstartdate', 'string'),
 ('noofhourspassedatfirstreaction', 'bigint'),
 ('result', 'struct<conclusion:string,sideeffectsonparticipant:string>')]

In [9]:
columns = ['ageofparticipant',
 'clinician.branch', 
 'clinician.name', 
 'clinician.role',
 'drug_used',
 'experimentenddate',
 'experimentstartdate',
 'noofhourspassedatfirstreaction',
 'result.conclusion',
 'result.sideeffectsonparticipant']

In [10]:
# To show column
trials_df.select(columns).show(5)

+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+
|ageofparticipant| branch|   name|     role|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|   conclusion|sideeffectsonparticipant|
+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+
|              19|Ontario|   Saul|therapist|  Placebo|    1619827200000|      1617235200000|                            52|BP normalized|          rashes on neck|
|              14|Ontario|   Saul|    nurse| Naproxen|    1619827200000|      1617235200000|                            78|    Follow-up|                     N/A|
|              17|Ontario|   Saul|    nurse|  Placebo|    1619827200000|      1617235200000|                            14|    Follow-up|                     N/A|
|              18|Onta

In [11]:
from pyspark.sql import functions as fn

In [12]:
trials_df.select([fn.count(fn.when(fn.col(column).isNull(),fn.col(column))) for column in columns])

DataFrame[count(CASE WHEN (ageofparticipant IS NULL) THEN ageofparticipant END): bigint, count(CASE WHEN (clinician.branch IS NULL) THEN clinician.branch END): bigint, count(CASE WHEN (clinician.name IS NULL) THEN clinician.name END): bigint, count(CASE WHEN (clinician.role IS NULL) THEN clinician.role END): bigint, count(CASE WHEN (drug_used IS NULL) THEN drug_used END): bigint, count(CASE WHEN (experimentenddate IS NULL) THEN experimentenddate END): bigint, count(CASE WHEN (experimentstartdate IS NULL) THEN experimentstartdate END): bigint, count(CASE WHEN (noofhourspassedatfirstreaction IS NULL) THEN noofhourspassedatfirstreaction END): bigint, count(CASE WHEN (result.conclusion IS NULL) THEN result.conclusion END): bigint, count(CASE WHEN (result.sideeffectsonparticipant IS NULL) THEN result.sideeffectsonparticipant END): bigint]

In [57]:
# counting null values per column

trials_df.select([fn.count(fn.when(fn.col(column).isNull(),column)).alias (column) for column in columns]).show() 

+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+
|ageofparticipant|clinician.branch|clinician.name|clinician.role|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|result.conclusion|result.sideeffectsonparticipant|
+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+
|               0|               0|             0|           109|        0|                0|                  0|                            73|               53|                              0|
+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+

